In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Veri setini yükleme ve hazırlama
df = pd.read_excel("rainfall-and-daily-consumption-data-on-istanbul-dams.xlsx")
df = df[['Tarih', 'İstanbul günlük tüketim(m³/gün)']]
df = df.set_index("Tarih")
df.index = pd.to_datetime(df.index)
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'] // 100
df['İstanbul günlük tüketim(m³/gün)'] = df['İstanbul günlük tüketim(m³/gün)'].astype(float)
df = np.log(df)

# Eğitim ve test setlerini oluşturma
train_size = int(len(df) * 0.80)
train, test = df.iloc[:train_size], df.iloc[train_size:]

# SARIMAX modelini oluşturma
order = (1, 1, 1)  # (p, d, q) değerleri
seasonal_order = (1, 1, 1, 7)  # (P, D, Q, s) değerleri
sarimax_model = SARIMAX(train['İstanbul günlük tüketim(m³/gün)'], order=order, seasonal_order=seasonal_order)
sarimax_model_fit = sarimax_model.fit(disp=False)
print(sarimax_model_fit.summary())

# Test seti üzerinde tahmin yapma
predictions = sarimax_model_fit.predict(start=test.index[0], end=test.index[-1])

# Hataları hesaplama
mse = mean_squared_error(test['İstanbul günlük tüketim(m³/gün)'], predictions)
print("Mean Squared Error (MSE): {:.4f}".format(mse))

rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE): {:.4f}".format(rmse))

mae = mean_absolute_error(test['İstanbul günlük tüketim(m³/gün)'], predictions)
print("Mean Absolute Error (MAE): {:.4f}".format(mae))

mape = mean_absolute_percentage_error(test['İstanbul günlük tüketim(m³/gün)'], predictions)
print("Mean Absolute Percentage Error (MAPE): {:.4f}".format(mape*100))


C:\Users\DOGUKAN\anaconda3\envs\tensorflow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\DOGUKAN\anaconda3\envs\tensorflow\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\DOGUKAN\anaconda3\envs\tensorflow\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                      SARIMAX Results                                      
Dep. Variable:     İstanbul günlük tüketim(m³/gün)   No. Observations:                 3771
Model:               SARIMAX(1, 1, 1)x(1, 1, 1, 7)   Log Likelihood                8233.132
Date:                             Tue, 30 Apr 2024   AIC                         -16456.264
Time:                                     20:46:53   BIC                         -16425.099
Sample:                                 01-01-2011   HQIC                        -16445.182
                                      - 04-28-2021                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5343      0.014     38.412      0.000       0.507       0.562
ma.L1         -0.9002      

In [3]:
predictions

2021-04-29    10.215629
2021-04-30    10.214520
2021-05-01    10.218218
2021-05-02    10.229681
2021-05-03    10.222533
                ...    
2023-11-23    10.191004
2023-11-24    10.187286
2023-11-25    10.193093
2023-11-26    10.206800
2023-11-27    10.195402
Freq: D, Name: predicted_mean, Length: 943, dtype: float64

In [4]:
pred8=predictions

In [5]:
len(pred8)

943

In [6]:
pred8=np.array(pred8)